<img src="finaldataframe.png" style="float: left; margin: 20px; height: 55px">

# Notebook 5- Compiling Final DataFrame

### _ Merging Seeking Alpha Analyses with Historical Stock Data_

---

### Notebook Summary
- The method for merging historical stock data with long and short idea analyses from Seeking Alpha will be utilizing Pandas "datetime" functionality to:
     - Create weekly offsets for one year (52 offsets) that will represent holding periods
     - Using the time of article posting as our starting point (aka offset 0 = time of article posting), engineering new columns by applying offsets to "Time"
     - Performing a merge for each week, which will append the appropriate stock data to the corresponding date and ticker 
---

In [1]:
# Importing necessary libraries 

import pandas as pd
import numpy as np 
from dateutil import parser
from datetime import datetime
from datetime import timedelta

#### Importing Final Stocks & Ideas DataFrames

In [2]:
stocks = pd.read_csv('final_stocks.csv')  # Exported from Notebook 3
ideas = pd.read_csv('final_ideas_df.csv') # Exported from Notebook 2

---
## Let's Check It Out!
---

In [3]:
ideas = ideas.rename(columns={'Time':'Date'}) #DROP THIS

#### & make sure there are no missing values

In [4]:
# ideas.isnull().sum() - No missing values
# stocks.isnull().sum() - No missing values

#### & check out the datatypes we are working with 

In [5]:
# ideas.dtypes
# stocks.dtypes

In [6]:
stocks = stocks[['Date','Close','Tickers']]

In [7]:
ideas = ideas[['Authors','Link','Tickers','Date','Title','Strategy']]

---
## Datatype Conversion 
_In order to properly interpret, analyze and model stock recommendations with historical stock pricing timeseries functionality, we must convert datatypes in order to merge the dataframes._



`Date` : Timestamp for each observation/date
- We will be relying heavily on Pandas DateTime for timeseries functionality 
- To do so, different holding periods need to be created with 'Date' as their starting point
- Historical stock data will be merged with these holding periods to investigate how accurate recommendations are when the dust settles 
- Creating holding periods aka `offsets` in  `weeklyincrements` 

---
### Converting columns to Pandas datetime objects
----

In [8]:
ideas['Date'] = pd.to_datetime(ideas['Date'])
stocks['Date'] = pd.to_datetime(stocks['Date'])

# Check progress

print(ideas['Date'].head(1))
print(stocks['Date'].head(1))

0   2017-12-31 21:16:00
Name: Date, dtype: datetime64[ns]
0   2017-01-03
Name: Date, dtype: datetime64[ns]


---
### Creating weekly offsets for 1 year
----
_Note : The largest increment timedelta can hold is days_

In [9]:
start = timedelta(days=0)

In [10]:
# Compiling list of weeks in terms of days (multiples of 7)
#
weeks = list(range(7,365,7)) 
#
# Compiling list of timedeltas for same range
offsets = list(pd.timedelta_range(start, periods=52, freq= '7D'))
#
# Iterating through both lists, adding each offset to time of article posting
for weeks_value, offsets_value in zip(weeks, offsets):
    ideas[weeks_value] = ideas['Date'] + offsets_value

---
### Converting new columns to datetime objects
----
_Note : Exclude categorical data and merge back after datetime transformation_

In [11]:
# Grabbing categorical columns
categorical = ideas[['Authors','Link','Tickers','Strategy','Title']]

In [12]:
# Creating lambda function to map datetime transformation to datetime columns
to_datetime = lambda x: pd.to_datetime(x)

In [13]:
# Dropping categorical columns so that applymap() can run
ideas.drop(['Authors', 'Link', 'Tickers', 'Title', 'Strategy'],axis=1,inplace=True)

# Applying functionn
ideas = ideas.applymap(to_datetime)

---
### Extracting week number for Ideas & Stocks
----

In [14]:
# Creatinng lambda function to map week accessor to datetime columns
to_week = lambda x: x.dt.week

In [15]:
# Applying function to Ideas DataFrame
for column in ideas:
    ideas[column] = ideas[column].dt.week

In [16]:
# Applying function to Stocks DataFrame 
stocks['Date'] = stocks['Date'].dt.week

In [17]:
# Joining categorical data back with Ideas DataFrame
ideas = categorical.join(ideas)

---
### Merging Ideas & Stocks DataFrames
----
- Left merge
- Align column names
- 52 merges represent 52 offsets created for 52 weeks out of the year

In [18]:
stocks = stocks.rename(columns={'Close':'Opening Price'})
opening = pd.merge(stocks,ideas,on=['Date','Tickers'])
opening.drop_duplicates(subset='Link',inplace=True)

In [19]:
week1 = opening.rename(columns={7:'7'})
stocks = stocks.rename(columns={'Opening Price':'Week1','Date':'7'})
week1 = pd.merge(stocks,week1,on=['7','Tickers'])
week1.drop_duplicates(subset='Link',inplace=True)

In [20]:
week2 = week1.rename(columns={14:'14'})
stocks = stocks.rename(columns={'Week1':'Week2','7':'14'})
week2 = pd.merge(stocks,week2,on=['14','Tickers'])
week2.drop_duplicates(subset='Link',inplace=True)

In [21]:
week3 = week2.rename(columns={21:'21'})
stocks = stocks.rename(columns={'Week2':'Week3','14':'21'})
week3 = pd.merge(stocks,week3,on=['21','Tickers'])
week3.drop_duplicates(subset='Link',inplace=True)

In [22]:
week4 = week3.rename(columns={28:'28'})
stocks = stocks.rename(columns={'Week3':'Week4','21':'28'})
week4 = pd.merge(stocks,week4,on=['28','Tickers'])
week4.drop_duplicates(subset='Link',inplace=True)

In [23]:
week5 = week4.rename(columns={35:'35'})
stocks = stocks.rename(columns={'Week4':'Week5','28':'35'})
week5 = pd.merge(stocks,week5,on=['35','Tickers'])
week5.drop_duplicates(subset='Link',inplace=True)

In [24]:
week6 = week5.rename(columns={42:'42'})
stocks = stocks.rename(columns={'Week5':'Week6','35':'42'})
week6 = pd.merge(stocks,week6,on=['42','Tickers'])
week6.drop_duplicates(subset='Link',inplace=True)

In [25]:
week7 = week6.rename(columns={49:'49'})
stocks = stocks.rename(columns={'Week6':'Week7','42':'49'})
week7 = pd.merge(stocks,week7,on=['49','Tickers'])
week7.drop_duplicates(subset='Link',inplace=True)

In [26]:
week8 = week7.rename(columns={56:'56'})
stocks = stocks.rename(columns={'Week7':'Week8','49':'56'})
week8 = pd.merge(stocks,week8,on=['56','Tickers'])
week8.drop_duplicates(subset='Link',inplace=True)

In [27]:
week9 = week8.rename(columns={63:'63'})
stocks = stocks.rename(columns={'Week8':'Week9','56':'63'})
week9 = pd.merge(stocks,week9,on=['63','Tickers'])
week9.drop_duplicates(subset='Link',inplace=True)

In [28]:
week10 = week9.rename(columns={70:'70'})
stocks = stocks.rename(columns={'Week9':'Week10','63':'70'})
week10 = pd.merge(stocks,week10,on=['70','Tickers'])
week10.drop_duplicates(subset='Link',inplace=True)

In [29]:
week11 = week10.rename(columns={77:'77'})
stocks = stocks.rename(columns={'Week10':'Week11','70':'77'})
week11 = pd.merge(stocks,week11,on=['77','Tickers'])
week11.drop_duplicates(subset='Link',inplace=True)

In [30]:
week12 = week11.rename(columns={84:'84'})
stocks = stocks.rename(columns={'Week11':'Week12','77':'84'})
week12 = pd.merge(stocks,week12,on=['84','Tickers'])
week12.drop_duplicates(subset='Link',inplace=True)

In [31]:
week13 = week12.rename(columns={91:'91'})
stocks = stocks.rename(columns={'Week12':'Week13','84':'91'})
week13 = pd.merge(stocks,week13,on=['91','Tickers'])
week13.drop_duplicates(subset='Link',inplace=True)

In [32]:
week14 = week13.rename(columns={98:'98'})
stocks = stocks.rename(columns={'Week13':'Week14','91':'98'})
week14 = pd.merge(stocks,week14,on=['98','Tickers'])
week14.drop_duplicates(subset='Link',inplace=True)

In [33]:
week15 = week14.rename(columns={105:'105'})
stocks = stocks.rename(columns={'Week14':'Week15','98':'105'})
week15 = pd.merge(stocks,week15,on=['105','Tickers'])
week15.drop_duplicates(subset='Link',inplace=True)

In [34]:
week16 = week15.rename(columns={112:'112'})
stocks = stocks.rename(columns={'Week15':'Week16','105':'112'})
week16 = pd.merge(stocks,week16,on=['112','Tickers'])
week16.drop_duplicates(subset='Link',inplace=True)

In [35]:
week17 = week16.rename(columns={119:'119'})
stocks = stocks.rename(columns={'Week16':'Week17','112':'119'})
week17 = pd.merge(stocks,week17,on=['119','Tickers'])
week17.drop_duplicates(subset='Link',inplace=True)

In [36]:
week18 = week17.rename(columns={126:'126'})
stocks = stocks.rename(columns={'Week17':'Week18','119':'126'})
week18 = pd.merge(stocks,week18,on=['126','Tickers'])
week18.drop_duplicates(subset='Link',inplace=True)

In [37]:
week19 = week18.rename(columns={133:'133'})
stocks = stocks.rename(columns={'Week18':'Week19','126':'133'})
week19 = pd.merge(stocks,week19,on=['133','Tickers'])
week19.drop_duplicates(subset='Link',inplace=True)

In [38]:
week20 = week19.rename(columns={140:'140'})
stocks = stocks.rename(columns={'Week19':'Week20','133':'140'})
week20 = pd.merge(stocks,week20,on=['140','Tickers'])
week20.drop_duplicates(subset='Link',inplace=True)

In [39]:
week21 = week20.rename(columns={147:'147'})
stocks = stocks.rename(columns={'Week20':'Week21','140':'147'})
week21 = pd.merge(stocks,week21,on=['147','Tickers'])
week21.drop_duplicates(subset='Link',inplace=True)

In [40]:
week22 = week21.rename(columns={154:'154'})
stocks = stocks.rename(columns={'Week21':'Week22','147':'154'})
week22 = pd.merge(stocks,week22,on=['154','Tickers'])
week22.drop_duplicates(subset='Link',inplace=True)

In [41]:
week23 = week22.rename(columns={161:'161'})
stocks = stocks.rename(columns={'Week22':'Week23','154':'161'})
week23 = pd.merge(stocks,week23,on=['161','Tickers'])
week23.drop_duplicates(subset='Link',inplace=True)

In [42]:
week24 = week23.rename(columns={168:'168'})
stocks = stocks.rename(columns={'Week23':'Week24','161':'168'})
week24 = pd.merge(stocks,week24,on=['168','Tickers'])
week24.drop_duplicates(subset='Link',inplace=True)

In [43]:
week25 = week24.rename(columns={175:'175'})
stocks = stocks.rename(columns={'Week24':'Week25','168':'175'})
week25 = pd.merge(stocks,week25,on=['175','Tickers'])
week25.drop_duplicates(subset='Link',inplace=True)

In [44]:
week26 = week25.rename(columns={182:'182'})
stocks = stocks.rename(columns={'Week25':'Week26','175':'182'})
week26 = pd.merge(stocks,week26,on=['182','Tickers'])
week26.drop_duplicates(subset='Link',inplace=True)

In [45]:
week27 = week26.rename(columns={189:'189'})
stocks = stocks.rename(columns={'Week26':'Week27','182':'189'})
week27 = pd.merge(stocks,week27,on=['189','Tickers'])
week27.drop_duplicates(subset='Link',inplace=True)

In [46]:
week28 = week27.rename(columns={196:'196'})
stocks = stocks.rename(columns={'Week27':'Week28','189':'196'})
week28 = pd.merge(stocks,week28,on=['196','Tickers'])
week28.drop_duplicates(subset='Link',inplace=True)

In [47]:
week29 = week28.rename(columns={203:'203'})
stocks = stocks.rename(columns={'Week28':'Week29','196':'203'})
week29 = pd.merge(stocks,week29,on=['203','Tickers'])
week29.drop_duplicates(subset='Link',inplace=True)

In [48]:
week30 = week29.rename(columns={210:'210'})
stocks = stocks.rename(columns={'Week29':'Week30','203':'210'})
week30 = pd.merge(stocks,week30,on=['210','Tickers'])
week30.drop_duplicates(subset='Link',inplace=True)

In [49]:
week31 = week30.rename(columns={217:'217'})
stocks = stocks.rename(columns={'Week30':'Week31','210':'217'})
week31 = pd.merge(stocks,week31,on=['217','Tickers'])
week31.drop_duplicates(subset='Link',inplace=True)

In [50]:
week32 = week31.rename(columns={224:'224'})
stocks = stocks.rename(columns={'Week31':'Week32','217':'224'})
week32 = pd.merge(stocks,week32,on=['224','Tickers'])
week32.drop_duplicates(subset='Link',inplace=True)

In [51]:
week33 = week32.rename(columns={231:'231'})
stocks = stocks.rename(columns={'Week32':'Week33','224':'231'})
week33 = pd.merge(stocks,week33,on=['231','Tickers'])
week33.drop_duplicates(subset='Link',inplace=True)

In [52]:
week34 = week33.rename(columns={238:'238'})
stocks = stocks.rename(columns={'Week33':'Week34','231':'238'})
week34 = pd.merge(stocks,week34,on=['238','Tickers'])
week34.drop_duplicates(subset='Link',inplace=True)

In [53]:
week35 = week34.rename(columns={245:'245'})
stocks = stocks.rename(columns={'Week34':'Week35','238':'245'})
week35 = pd.merge(stocks,week35,on=['245','Tickers'])
week35.drop_duplicates(subset='Link',inplace=True)

In [54]:
week36 = week35.rename(columns={252:'252'})
stocks = stocks.rename(columns={'Week35':'Week36','245':'252'})
week36 = pd.merge(stocks,week36,on=['252','Tickers'])
week36.drop_duplicates(subset='Link',inplace=True)

In [55]:
week37 = week36.rename(columns={259:'259'})
stocks = stocks.rename(columns={'Week36':'Week37','252':'259'})
week37 = pd.merge(stocks,week37,on=['259','Tickers'])
week37.drop_duplicates(subset='Link',inplace=True)

In [56]:
week38 = week37.rename(columns={266:'266'})
stocks = stocks.rename(columns={'Week37':'Week38','259':'266'})
week38 = pd.merge(stocks,week38,on=['266','Tickers'])
week38.drop_duplicates(subset='Link',inplace=True)

In [57]:
week39 = week38.rename(columns={273:'273'})
stocks = stocks.rename(columns={'Week38':'Week39','266':'273'})
week39 = pd.merge(stocks,week39,on=['273','Tickers'])
week39.drop_duplicates(subset='Link',inplace=True)

In [58]:
week40 = week39.rename(columns={280:'280'})
stocks = stocks.rename(columns={'Week39':'Week40','273':'280'})
week40 = pd.merge(stocks,week40,on=['280','Tickers'])
week40.drop_duplicates(subset='Link',inplace=True)

In [59]:
week41 = week40.rename(columns={287:'287'})
stocks = stocks.rename(columns={'Week40':'Week41','280':'287'})
week41 = pd.merge(stocks,week41,on=['287','Tickers'])
week41.drop_duplicates(subset='Link',inplace=True)

In [60]:
week42 = week41.rename(columns={294:'294'})
stocks = stocks.rename(columns={'Week41':'Week42','287':'294'})
week42 = pd.merge(stocks,week42,on=['294','Tickers'])
week42.drop_duplicates(subset='Link',inplace=True)

In [61]:
week43 = week42.rename(columns={301:'301'})
stocks = stocks.rename(columns={'Week42':'Week43','294':'301'})
week43 = pd.merge(stocks,week43,on=['301','Tickers'])
week43.drop_duplicates(subset='Link',inplace=True)

In [62]:
week44 = week43.rename(columns={308:'308'})
stocks = stocks.rename(columns={'Week43':'Week44','301':'308'})
week44 = pd.merge(stocks,week44,on=['308','Tickers'])
week44.drop_duplicates(subset='Link',inplace=True)

In [63]:
week45 = week44.rename(columns={315:'315'})
stocks = stocks.rename(columns={'Week44':'Week45','308':'315'})
week45 = pd.merge(stocks,week45,on=['315','Tickers'])
week45.drop_duplicates(subset='Link',inplace=True)

In [64]:
week46 = week45.rename(columns={322:'322'})
stocks = stocks.rename(columns={'Week45':'Week46','315':'322'})
week46 = pd.merge(stocks,week46,on=['322','Tickers'])
week46.drop_duplicates(subset='Link',inplace=True)

In [65]:
week47 = week46.rename(columns={329:'329'})
stocks = stocks.rename(columns={'Week46':'Week47','322':'329'})
week47 = pd.merge(stocks,week47,on=['329','Tickers'])
week47.drop_duplicates(subset='Link',inplace=True)

In [66]:
week48 = week47.rename(columns={336:'336'})
stocks = stocks.rename(columns={'Week47':'Week48','329':'336'})
week48 = pd.merge(stocks,week48,on=['336','Tickers'])
week48.drop_duplicates(subset='Link',inplace=True)

In [67]:
week49 = week48.rename(columns={343:'343'})
stocks = stocks.rename(columns={'Week48':'Week49','336':'343'})
week49 = pd.merge(stocks,week49,on=['343','Tickers'])
week49.drop_duplicates(subset='Link',inplace=True)

In [68]:
week50 = week49.rename(columns={350:'350'})
stocks = stocks.rename(columns={'Week49':'Week50','343':'350'})
week50 = pd.merge(stocks,week50,on=['350','Tickers'])
week50.drop_duplicates(subset='Link',inplace=True)

In [69]:
week51 = week50.rename(columns={357:'357'})
stocks = stocks.rename(columns={'Week50':'Week51','350':'357'})
week51 = pd.merge(stocks,week51,on=['357','Tickers'])
week51.drop_duplicates(subset='Link',inplace=True)

In [70]:
week52 = week51.rename(columns={364:'364'})
stocks = stocks.rename(columns={'Week51':'Week52','357':'364'})
week52 = pd.merge(stocks,week52,on=['364','Tickers'])
week52.drop_duplicates(subset='Link',inplace=True)

In [71]:
# Reordering columns for readability 

final = week52[['Authors','Link','Title','Strategy','Tickers', 'Week1',
        'Week2','Week3','Week4','Week5','Week6','Week7','Week8','Week9','Week10','Week11',
       'Week12','Week13','Week14','Week15','Week16','Week17','Week18', 'Week19','Week20',
       'Week21','Week22','Week23','Week24','Week25','Week26','Week27','Week28','Week29',
       'Week30', 'Week31','Week32','Week33','Week34','Week35','Week36','Week37','Week38',
       'Week39','Week40','Week41','Week42','Week43', 'Week44', 'Week45','Week46','Week47',
       'Week48','Week49','Week50','Week51','Week52']]

In [72]:
final.head(1)

,Authors,Link,Title,Strategy,Tickers,Week1,Week2,Week3,Week4,Week5,...,Week43,Week44,Week45,Week46,Week47,Week48,Week49,Week50,Week51,Week52
0,Paulo Santos,/article/4038275-apple-unexpected-positive-app...,Apple: An Unexpected Positive Appears,Long,AAPL,120.0,120.080002,121.629997,130.289993,133.289993,...,174.25,173.970001,169.979996,174.089996,169.800003,172.669998,176.419998,170.570007,116.150002,118.989998


---
## Accounting For Actual Performace 
_In order to properly interpret, analyze and model stock recommendations with historical stock pricing timeseries functionality, we must convert datatypes in order to merge the dataframes._

---
- Performance after 1 month, 3 months, 6 months, and 1 year

In [87]:
final2 = final.copy()

In [80]:
final['1 Month Hold'] = final['Week4'] - final['Week1']
final['3 Month Hold'] = final['Week12'] - final['Week1']
final['6 Month Hold'] = final['Week24'] - final['Week1']
final['12 Month Hold'] = final['Week52'] - final['Week1']

/Users/laurencable/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/laurencable/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/laurencable/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

In [81]:
# Creating column represening difference in pricing 3 months after opening
final['1 Month Hold'] = final['Week4'] - final['Week1']
final['3 Month Hold'] = final['Week12'] - final['Week1']
final['6 Month Hold'] = final['Week24'] - final['Week1']
final['12 Month Hold'] = final['Week52'] - final['Week1']

# If increased, apply 0
# If decreased, apply 1
final['1 Month Hold'] = final['1 Month Hold'].apply(lambda x: 0 if x > 0.0 else 1)
final['3 Month Hold'] = final['3 Month Hold'].apply(lambda x: 0 if x > 0.0 else 1)
final['6 Month Hold'] = final['6 Month Hold'].apply(lambda x: 0 if x > 0.0 else 1)
final['12 Month Hold'] = final['12 Month Hold'].apply(lambda x: 0 if x > 0.0 else 1)

/Users/laurencable/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/laurencable/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/laurencable/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

In [84]:
final = final[['Link','Strategy','Tickers','1 Month Hold','3 Month Hold','6 Month Hold','12 Month Hold']]

In [86]:
final = final.reset_index(drop=True)

In [92]:
final.to_csv('tuesday.csv')

In [93]:
final.head(1)

,Link,Strategy,Tickers,1 Month Hold,3 Month Hold,6 Month Hold,12 Month Hold
0,/article/4038275-apple-unexpected-positive-app...,Long,AAPL,0,0,0,1


---
## Saving Results 

---

#### Exporting as CSV  
- Save `Final` `DataFrame` csv file
- Read csv back in to ensure export was successful

In [ ]:
# Exporting as csv
week52.to_csv('final_dataframe.csv',index=False)

# Reading csv back in 
week52 = pd.read_csv('final_dataframe.csv')

# Inspecting head
final_dataframe.head(2)

## Onwards!

---

## Please proceed to Notebook 6 :)  